# Day 12: Hill Climbing Algorithm

## Part 1

In [ ]:
from aoc_2023 import core


_example = """Sabqponm
abcryxxl
accszExk
acctuvwj
abdefghi"""
_test = core.read_input("../data/day_12.txt")


Position = tuple[int, int]

In [ ]:
import string


def parse(s: str) -> tuple[int, int, list[list[int]]]:
    heights = {
        letter: i
        for i, letter in enumerate(string.ascii_lowercase)
    }

    start: tuple[int, int] | None = None
    finish: tuple[int, int] | None = None
    
    lines = s.split("\n")
    topo_map: list[list[int | None]] = [
        [None for _ in range(len(line))]
        for line in lines
    ]

    for i, line in enumerate(lines):
        for j, char in enumerate(line):
            if char == "S":
                start = (i, j)
                topo_map[i][j] = heights["a"]
            elif char == "E":
                finish = (i, j)
                topo_map[i][j] = heights["z"]
            else:
                topo_map[i][j] = heights[char]

    return start, finish, topo_map                

In [ ]:
def dijkstra(graph: dict[Position, set[Position]], a: Position, b: list[Position]) -> int:
    dist = {v: 0 if v == a else None for v in graph}
    prev = {v: None for v in graph}

    unvisited = set(graph)
    
    while unvisited:
        candidates = [
            (dist[v], v)
            for v in unvisited
            if dist[v] is not None
        ]
        if not candidates:
            break
        _, current = min(candidates)
        unvisited.remove(current)

        for neighbor in graph[current]:
            if neighbor in unvisited:
                alt = dist[current] + 1
                if dist[neighbor] is None or alt < dist[neighbor]:
                    dist[neighbor] = alt
                    prev[neighbor] = current

    return min(dist[v] for v in b if dist[v] is not None)

In [ ]:
def to_graph(topo_map: list[list[int]]) -> dict[Position, set[Position]]:
    m = len(topo_map)
    n = len(topo_map[0])
    
    graph = {(i, j): set() for i in range(m) for j in range(n)}
    
    for i, row in enumerate(topo_map):
        for j, height in enumerate(row):
            # Reverse direction in the graph to transform the problem
            # from having multiple starts (Part 2) to having multiple finishes
            if i > 0 and (topo_map[i - 1][j] <= height + 1):
                graph[(i - 1, j)].add((i, j))
            if i < m - 1 and (topo_map[i + 1][j] <= height + 1):
                graph[(i + 1, j)].add((i, j))
            if j > 0 and (topo_map[i][j - 1] <= height + 1):
                graph[(i, j - 1)].add((i, j))
            if j < n - 1 and (topo_map[i][j + 1] <= height + 1):
                graph[(i, j + 1)].add((i , j))
                
    return graph

In [ ]:
def part_1(s: str) -> int:
    start, finish, topo_map = parse(s)
    graph = to_graph(topo_map)
    
    # We are reversing the direction of the graph, so
    # our starts and finish need to be swapped.
    return dijkstra(graph, finish, [start])

In [ ]:
part_1(_example)

31

In [ ]:
part_1(_test)

449

## Part 2

In [ ]:
def part_2(s: str) -> int:
    start, finish, topo_map = parse(s)
    graph = to_graph(topo_map)
    candidates = [(i, j) for i in range(len(topo_map)) for j in range(len(topo_map[i])) if topo_map[i][j] == 0]

    # Because we are reversing the direction of the graph
    # we can treat our candidates as multiple finishes instead of starts
    # and calculate the distance in a single run of Dijkstra.
    return dijkstra(graph, finish, candidates)

In [ ]:
part_2(_example)

29

In [ ]:
part_2(_test)

443